In [1]:
!pip install scipy
!pip install matplotlib

from ipywidgets import interactive, widgets
from IPython.display import display

from IPython.utils import io

import math
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

<b>Statistical OEE Measurement</b>

Overall Equipment Effectiveness (OEE) is the product of Availability (equipment uptime), Productivity (cycle time or capacity), and Quality (process yield) as compared to reference values. The reference values are typically taken from the Systems Requirements Specification (SRS) or project charter. At nominal availability, productivity, and quality, OEE is 1 (equivalently expressed as 100%). 

When validating a system or machine for Site Acceptance Testing (SAT), the system is run for a fixed duration to obtain sample data. In theory, the sample data can be statistically analyzed to produce a confidence interval for the true OEE of the system, which can then be compared against target OEE values to determine whether the system passes SAT. In reality, while Quality and Productivity may be analyzed in this way, the sensitivity of Availability to infrequent events and its correlation to long-term, seasonal variability makes Availability extremely difficult to model statistically, especially in the context of SAT where historical data are not available. The difficulties of predicting Availability are discussed in further detail at the end of this article, but a statistical solution is not presented. 

With regards to Quality and Productivity and the statistical analysis of SAT, more data provides greater confidence in SAT results, but more data requires more time and resources to collect. Thus, maximizing SAT result confidence and minimizing required SAT time are competing requirements. This article seeks to establish a quantitative relationship between target confidence level and the amount of data required, which should allow SAT planners to control OEE measurement accuracy while optimizing SAT duration. 

<b>Measuring Quality</b>

Quality can be modeled as a binomial random variable (pass or fail), and the one-sided confidence interval can be given by tables or approximated using the Wilson score interval as follows:
$$
\begin{aligned}
p \approx \left(\frac{1}{n + z^2}\left( s+\frac{1}{2}z^2 - z \sqrt{\frac{sf}{n}+\frac{z^2}{4}} \right),\: 1\right]
\end{aligned}
$$
where $-p$ is the bound of interest, since we would like Quality to be <i>above</i> a particular value; $n$ is the sample size; $s$ is the number of observed successes; $f$ is the number of observed failures (such that $f = n - s$); and $z$ is the probit, i.e. the one-sided quantile of the normal distribution corresponding to a confidence level $c$. For example, for a 95% confidence level, $z \approx 1.64$. 

An interactive calculator is implemented below.

In [5]:
# wilson score interval
def wilson_ci(c=0.95, m=0.95, n=1000):
    (s, f) = (m*n, n - m*n) # successes, failures
    z = stats.norm.ppf(c) # one-sided normal probit

    p = (((s + 0.5*z**2) - z*((s*f)/n + z**2/4)**0.5)/(n + z**2), 1)

    print("The %2d%% confidence interval is (%4.3f, %4.3f)." % (c*100, p[0], p[1]))
    
    return p

In [6]:
w = interactive(
    wilson_ci, 
    c = widgets.FloatSlider(min=0.70, max=0.99, value=0.95, step=0.01, description="confidence level"),
    m = widgets.FloatSlider(min=0.50, max=1.00, value=0.95, step=0.01, description="sample yield"), 
    n = widgets.IntSlider(min=50, max=5000, value=1000, step=50, description = "sample size"))
w

interactive(children=(FloatSlider(value=0.95, description='confidence level', max=0.99, min=0.7, step=0.01), F…

As shown by the above demonstration, increasing the sample size reduces measurement uncertainty (decreases the size of the confidence interval). Also, increasing the sample yield reduces measurement uncertainty (since yield is known to be bounded at 100%), whereas moving the sample yield toward 50% increases the measurement uncertainty. Thus, to maximize confidence when measuring Quality, the sample size should be maximized, especially if the yield is low. If the sample yield is very high (99% or greater), a lower sample size can achieve the same level of confidence. 

<b>Measuring Productivity</b>

Productivity can be quantified as a function of cycle time, and cycle time can be modeled as a normally-distributed random variable. Thus, the one-sided confidence interval can be calculated using the Student t distribution as follows.
$$
\begin{aligned}
p = \left(0,\: \mu + t_{\alpha,n-1}\frac{\sigma}{\sqrt{n}}\right)
\end{aligned}
$$
where the upper bound $+p$ is the bound of interest (we want confidence that the CT is <i>below</i> a certain value); $t$ is the <i>t</i>-statistic; $\alpha$ is the one-sided target error (i.e. $\alpha = 1-c$ where $c$ is the confidence level); $n$ is the sample size; $\mu$ is the sample mean; and $\sigma$ is the sample standard deviation, i.e. $\sigma^2$ is the sample variance. 

In [5]:
# student t confidence interval
def norm_ci(c=0.95, ct=2.0, m=2.0, s=0.4, n=1000):
    t = stats.t.ppf(c, n-1) # one-sided t-statistic

    p = (0, m + t*s/(n**0.5))

    print("The %2d%% confidence interval is (%4.3f, %4.3f)." % (c*100, ct/p[1], 1))
    
    def fnorm(x):
        return (1/(s*(2*math.pi)**0.5))*math.exp(-0.5*((x-m)/s)**2)
    
    x = np.linspace(m-5*s, m+5*s, 100)
    pdf = np.vectorize(fnorm)(x)
    
    plt.plot(x, pdf)
    plt.plot([ct, ct], [0, max(pdf)], 'r')
    plt.show()
    
    return (ct/p[1], 1)

In [6]:
z = interactive(
    norm_ci, 
    c = widgets.FloatSlider(min=0.70, max=0.99, value=0.95, step=0.01, description="confidence level"),
    ct = widgets.FloatSlider(min=1.0, max=3.0, value=2.0, step=0.02, description="target CT"),
    m = widgets.FloatSlider(min=1.0, max=3.0, value=2.0, step=0.02, description="sample CT"), 
    s = widgets.FloatSlider(min=0.1, max=1.0, value=0.4, step=0.1, description="sample stdev"),
    n = widgets.IntSlider(min=50, max=1000, value=1000, step=50, description = "sample size"))
z

interactive(children=(FloatSlider(value=0.95, description='confidence level', max=0.99, min=0.7, step=0.01), F…

Just as with the Quality measurement, increasing the sample size reduces measurement uncertainty of the Productivity measurment as well. Decreasing the sample standard deviation also decreases the measurement uncertainty, although this is a property of the process and is not directly controllable during the SAT setup. Note that because Productivity compares cycle time against a target value (typically defined in the SRS), if the measured CT outperforms the target CT, the Productivity parameter may exceed 1. 

Note also that this model is an approximation with known limitations. For instance, the cycle time cannot be negative. When the sample CT is large and the sample $\sigma$ is small, then this is a good approximation. However, if $\mu - 2\sigma \leq 0$, then the cycle time distribution is most likely not normal (specifically, left skewed) and the above results will be invalid. For an automated process, this should be uncommon. 

<b>Measuring Availability</b>

Availability is the hardest metric to measure. Availability can be understood as a time average of machine status, where the machine status takes a boolean value ("operational/up" or "unoperational/down"). Availablity can be estimated using Reliability Block Diagrams or Fault Tree Analysis, but both are highly dependent on subjective brainstorming and specific expertise, and are not strictly empirical methods. Empirical methods for forecasting these types of infrequent events is difficult in general (e.g. research on using a two-state Markov-chain to model the Clear Sky Index for solar irradiance), and impossible without abundant historical data; thus, forecasting and/or measuring of equipment availability during a SAT run of a new machine is impossible. 

The impossibility of measuring availability using a SAT run can be illustrated as follows. Let us assume that failure events follow a Poisson distribution (memoryless). As a general property, if a Poisson event has a mean frequency of 1 over a given period, the probability that the event never occurs over that same period is roughly 37%. Applied to our example, if we expect the machine to fail on average once per day, and we perform a SAT run which lasts one day, then there is a 37% chance that no failure occurs during the SAT run. From the opposite perspective, if no failure occurs during the SAT run, one can conclude with at most 63% confidence that the long-term failure rate will be less than once per day. This is dramatically worse than the typical 95% confidence standard applied to general statistical inferences. If the SAT run is shorter, say 5 hours, or half a shift, then a failure-free SAT run would give at most 63% confidence that the long-term failure rate will be less than once per 5 hours; in other words, even if the system demonstrates a perfect 5-hour run, there is still at least 37% percent chance that, in the long-run, the system will fail more than once per 5 hours! 

Furthermore, measuring availability requires not only the measurement of failure frequency, but also of the Mean Time To Repair (MTTR). Even if the TTR of failure events is a normally distributed random variable (a na&iuml;ve assumption), a statistical sample of at least 30 failures must be observed to make a meaningful measurement. This is obviously not practical during SAT. 

In conclusion, while Availability can be a significant contributor to OEE, it cannot be empirically measured during a SAT run. Therefore, alternative methods such as Fault Tree Analysis or a simple aggregate estimate must be employed to obtain a realistic result for the system's OEE. However, one must be careful about choosing Availability definitions when factoring Availability into SAT performance. Overall Availability is defined simply as downtime over total time, where downtime includes unscheduled maintenance, scheduled (preventive) maintenance, schedule loss, startup and shift changes, etc. Inherent Availability includes only downtime caused by unscheduled maintenance, and does not include downtime that is not caused directly by the system itself. When assessing a system for SAT, Inherent Availability should be the relevant parameter, but one must note that the Overall Availability observed over long-term usage of the system will necessarily be lower than the Inherent Availability, and how this impacts factory planning must be further scrutinized. 

A one-sided confidence interval for the for the true mean of a Poisson distribution can be obtained as follows:
$$
\begin{aligned}
p = \left[0,\: \frac{1}{2}\chi^2\left(c, 2k+2\right)\right)
\end{aligned}
$$
where the upper bound $+p$ is the bound of interest (we want confidence that the failure rate is <i>below</i> a certain value); $\chi^2(a,b)$ is the quantile function of a chi-squared distribution with significance level $a$ and $b$ degrees of freedom; $c$ is the confidence level; and $k$ is the number of observed failures in the observation period. 


The Availability metric depends not only on the failure rate, but also the Mean Time To Repair (MTTR), which can be understood as the mean downtime per failure. This depends primarily on the nature of the failure, whether spares are readily available, etc. The MTTR is itself a random variable, which can be modeled as a normally-distributed random variable, since no other information is known/assumed. However, because different failure modes may have vastly different MTTRs, a normal distribution is unlikely to be a good model. 

If the MTTR is known, the Availability can be given by:
$$
\begin{aligned}
\text{Availability} = 1 - \frac{\text{MTTR} \times \text{Failure Rate}}{T}
\end{aligned}
$$
where $T$ is a reference length of time, e.g. the length of one shift; and the failure rate is given as the mean number of failures per time $T$. 

In [5]:
def poisson_ci(c=0.95, k=0, t=1, MTTR=0.5):
    p = (0, 0.5*stats.chi2.ppf(c,2*k+2))
    
    A = 1 - (MTTR*p[1]/t)
    
    shiftlen = 10 # hours
    print("With %2d%% confidence, the failure rate will be less than %4.2f per shift." % (c*100, p[1]*shiftlen/t))
    print("With at most %2d%% confidence, the Availability will be at least %4.3f." % (c*100, A))
    
    return p

In [6]:
g = interactive(
    poisson_ci, 
    c = widgets.FloatSlider(min=0.70, max=0.99, value=0.95, step=0.01, description="confidence level"),
    k = widgets.IntSlider(min=0, max=20, value=0, step=1, description="observed failures"),
    t = widgets.IntSlider(min=1, max=50, value=1, step=1, description="observation time (hrs)"),
    MTTR = widgets.FloatSlider(min=0.1, max=2, value=0.5, step=0.1, description="MTTR"))
g

interactive(children=(FloatSlider(value=0.95, description='confidence level', max=0.99, min=0.7, step=0.01), I…

<b>Measuring OEE</b>

Aggregating the above results gives our statistical OEE measurement. As explained earlier, Availability will not be included in this aggregate, but should be taken into account using other means. Any Availability estimate should then be used to derate the OEE presented here to give a more realistic OEE result. 

Taking a partial (excludes Availability) OEE as
$$
\begin{aligned}
OEE_{Q,P} = Quality \times Performance
\end{aligned}
$$
the result is as follows:

In [13]:
c = w.kwargs["c"]
n = w.kwargs["n"]

def oee_ci(c=w.kwargs["c"], n=w.kwargs["n"]):
    with io.capture_output() as captured:
        p_Q = wilson_ci(c, w.kwargs["m"], n)
    with io.capture_output() as captured:
        p_P = norm_ci(c, z.kwargs["ct"], z.kwargs["m"], z.kwargs["s"], n)
    
    p = (p_Q[0] * p_P[0], p_Q[1] * p_P[1])
    print("The OEE %2d%% confidence interval is (%4.3f, %4.3f)." % (c*100, p[0], p[1]))
    print("The size of the OEE confidence interval is %4.3f." % (p[1] - p[0]))
    
    return p

oee = interactive(
    oee_ci, 
    c = widgets.FloatSlider(min=0.70, max=0.99, value=w.kwargs["c"], step=0.01, description="confidence level"),
    n = widgets.IntSlider(min=50, max=5000, value=w.kwargs["n"], step=50, description = "sample size"))
oee

interactive(children=(FloatSlider(value=0.95, description='confidence level', max=0.99, min=0.7, step=0.01), I…

NB: The above result uses the sample aggregate data (mean, standard deviation, etc.) from the above sections. To change those data, the values must be changed in the above sliders. 

If the lower bound $-p$ of the confidence interval meets the SAT requirement, then there is at least a $\frac{1-c}{2}$ chance that the true OEE meets the SAT requirement, where $c$ is the confidence level. If the lower bound fails the SAT requirement, then the confidence level can be adjusted in the above sliders until the lower bound rises above the SAT requirement. The chance that the true OEE meets the SAT requirement can then be calculated (as $\frac{1-c}{2}$) using this lower confidence level. 

As shown in this article, the required confidence level depends upon the performance of the system during SAT. If the system significantly exceeds expectations, then the confidence level required to ensure conformity is lower. On the other hand, if the system performance during SAT hovers around or just barely exceeds SAT requirements, then the confidence level required to ensure conformity will be very high. In this latter case, a short SAT run may not generate enough data to meet the required confidence level, so the SAT run may be extended, or a follow-up SAT run may be performed to generate more data and increase the confidence level. Thus, this article presents a framework for dynamically adjusting the length of the SAT run to ensure the system performs to specification. 